# Bug Detection

In this notebook I will show the baseline model for the bug detection project. This will be the simplest model that can be created to try and solve the problem of bug detection. The main idea of this model is that for each token of code we can predict if it is buggy or not. However we will be using multi class labels where each bug is an actual error class.

## Imports

In [1]:
import pickle
import codenet

import numpy as np
from tqdm import tqdm

from gensim.models import FastText

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

models_path = "../models/"

vectorizer_path = models_path + "vectorizer_fasttext.model"
classifier_path = models_path + "classifier_decisiontree.pkl"

## Data

In this section we load the data generated from the CodeNet subset we explored previously. The training data will consist of tokens of code as input and the respective error class as target. From the data exploration section we know that an error can be caused by a sequence of tokens, or an instruction. To label each token we will create an array of labels, each corresponding to a token. Then for each error class, if it's range includes a token we mark it with that label. In the end, the data will be a sequence of token, error class pairs. We want our model to be able to learn which token corresponds to which error class. The limitation of this method is the fact that we will not consider the context of the problem. We will consider each token to be independent of each other.

In [2]:
def convert_X_y(X, y):
    target_y = []
    
    for x, errs in zip(X, y):
        new_y = np.empty_like(x)
        new_y.fill("Accepted")
        for [i1, i2, err] in errs:
            new_y[i1:max(i1+1, i2)] = err
        target_y.append(new_y)
        
    return X, target_y

with open(codenet.detection_X_y_v2_path, 'rb') as f:
    X, ys = pickle.load(f)
    X, y = convert_X_y(X, ys)

Here we split the data into training and testing subsets with 80% of the sample for training and the rest for testing. Then we use FastText to vectorize the tokens into float arrays with size 16. I chose FastText because it handles words that are also out of vocabulary, like for example variable names that can be unique to a specific submission. Then we transform the input data from a sequence of tokens to an array containing the vectors for each token. The resulting data will be a sequence of vector, label pairs.

In [6]:
src_train, src_test, ys_train, ys_test = train_test_split(X, ys, train_size=0.8, random_state=42)

In [7]:
src_train, yss_train = convert_X_y(src_train, ys_train)
src_test, yss_test = convert_X_y(src_test, ys_test)

In [4]:
sentences = [x.tolist() for x in src_train]

vectorizer = FastText(sentences=sentences, min_count=0, vector_size=16, seed=42)

In [12]:
vectorizer.save(vectorizer_path)

In [8]:
tok_train = np.concatenate(src_train)
tok_test = np.concatenate(src_test)

X_train = vectorizer.wv[tok_train]
X_test = vectorizer.wv[tok_test]
y_train = np.concatenate(yss_train)
y_test = np.concatenate(yss_test)

print(f"Percent of accepted tokens in the test set {np.sum(y_test == 'Accepted') / len(y_test)}")

Percent of accepted tokens in the test set 0.8736134751773049


## Model

The baseline classifier we are going to use is a decision tree classifier.

In [6]:
clf = DecisionTreeClassifier(random_state=42)

In [7]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [8]:
y_pred = clf.predict(X_test)

In [9]:
print("Token Level Prediction")
print(f"Accuracy {accuracy_score(y_test, y_pred)}")
print(f"Precision {precision_score(y_test, y_pred, average='weighted', zero_division=0)}")
print(f"Recall {recall_score(y_test, y_pred, average='weighted', zero_division=0)}")
print(f"F1 {f1_score(y_test, y_pred, average='weighted', zero_division=0)}")

Accuracy 0.8728657548125633
Precision 0.787925806136263
Recall 0.8728657548125633
F1 0.8151843545926623


In [6]:
loop = tqdm(list(zip(src_test, yss_test)))

tp = 0
fp = 0
fn = 0
tn = 0

for i, (x_src, y_ref) in enumerate(loop):
    x_src = vectorizer.wv[x_src]
    y_p = clf.predict(x_src)
    
    ref_accepted = np.all(y_ref == "Accepted")
    p_accepted = np.all(y_p == "Accepted")
    
    if ref_accepted and p_accepted:
        tp += 1
    if not ref_accepted and p_accepted:
        fp += 1
    if ref_accepted and not p_accepted:
        fn += 1
    if not ref_accepted and not p_accepted:
        tn += 1

print("Document Level Prediction: Accepted vs non Accepted")
print(f"Accuracy {(tp + tn) / (tp + fp + fn + tn)}")
print(f"Precision {tp / (tp + fp)}")
print(f"Recall {tp / (tp + fn)}")
print(f"F1 {(2 * tp) / (2 * tp + fp + fn)}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19508/19508 [00:11<00:00, 1627.57it/s]

Document Level Prediction: Accepted vs non Accepted
Accuracy 0.515173262251384
Precision 0.5086231430360294
Recall 0.9154799713144145
F1 0.6539334065129894


In [24]:
loop = tqdm(list(zip(src_test, ys_test)))

tp = 0
fp = 0
fn = 0
tn = 0

for i, (x_src, y_ref) in enumerate(loop):
    x_src = vectorizer.wv[x_src]
    y_p = clf.predict(x_src)
    
    start_i = 0
    for i1, i2, err in y_ref:
        if np.all(y_p[start_i:i1] == "Accepted"):
            tp += 1
        else:
            fp += 1
        
        if np.all(y_p[i1:i2] == err):
            tn += 1
        else:
            fn += 1
            
        start_i = i2
    
    if y_p.shape[0] > start_i:
        if np.all(y_p[start_i:] == "Accepted"):
            tp += 1
        else:
            fp += 1

print("Statement Level Prediction: Accepted vs non Accepted")
print(f"Accuracy {(tp + tn) / (tp + fp + fn + tn)}")
print(f"Precision {tp / (tp + fp)}")
print(f"Recall {tp / (tp + fn)}")
print(f"F1 {(2 * tp) / (2 * tp + fp + fn)}")

100%|██████| 19508/19508 [00:12<00:00, 1621.32it/s]

Statement Level Prediction: Accepted vs non Accepted
Accuracy 0.6681046977576947
Precision 0.9797950262430737
Recall 0.6409246100437057
F1 0.7749332223262914


In [18]:
with open(classifier_path, "wb") as f:
    pickle.dump(clf, f)

## Inference

To perform inference we will take as input the Python source code, or the source file path, and tokenize it. We will thus obtain a sequence of all the tokens, their sequence number, line and column. Then we take the tokens and use our vectorizer to extract the features and prepare the data for inference. Finally we can run the model, the decision tree baseline, and obtain the list of errors for each token in our source code. We filter out the tokens without errors and only show the ones which have associated errors to them. To check the model we can also run the source code file and see what output we get.

In [4]:
vectorizer = FastText.load(vectorizer_path)

In [5]:
with open(classifier_path, "rb") as f:
    clf = pickle.load(f)

In [38]:
def inferbug_python(source_code):
    token_df = codenet.run_pythontokenizer_str(source_code)
    tokens = token_df["text"].values
    tokens = vectorizer.wv[tokens]

    prediction = clf.predict(tokens)
    token_df["prediction"] = prediction
    token_err_df = token_df[token_df["prediction"] != "Accepted"]
    
    return token_err_df

In [39]:
bug = """mylist = [1, 2, 3, double]

print(mylist[123])
"""

token_err_df = inferbug_python(bug)

display(token_err_df)
display(err_df2list(token_err_df))
codenet.exec_python_str(bug)

,seqnr,start,stop,text,class,channel,line,column,prediction
14,14,19,24,double,name,0,1,19,SyntaxError


[(14, 14, 'SyntaxError')]

('',
 'Traceback (most recent call last):\n  File "<string>", line 1, in <module>\nNameError: name \'double\' is not defined\n',
 1)